In [2]:
import gym
from collections import deque, namedtuple
import random
import numpy as np
from tqdm.notebook import tqdm 
import time
import altair as alt

import pandas as pd
import torch
import torch
from torch import nn
from torch.optim import SGD, Adam
from torch.distributions.categorical import Categorical

## Vanilla Policy Gradient

In [27]:
def create_model(input_size, output_size, hidden_layers=[16], activate_every=2):
    layers = [nn.Linear(in_shape, out_shape) for (in_shape, out_shape) in toolz.sliding_window(2, hidden_layers)]
    layers = [*toolz.concat([*ls, nn.ReLU()] for ls in toolz.partition(activate_every, layers))]
    layers = [nn.ReLU(), *layers]
    return nn.Sequential(
        nn.Linear(input_size, hidden_layers[0]),
        *layers,
        nn.Linear(hidden_layers[-1], output_size)
    )

def reward_to_go(rews):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
    return rtgs

def reward_to_go(rews):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
    return rtgs

class Agent():
    def __init__(self, env, lr=1e-2, batch_size=5000):
        self.env = env
        self.memory = self._create_memory()
        self.model = create_model(self.env.observation_space.shape[0], self.env.action_space.n)
        self.optim = Adam(self.model.parameters(), lr=lr)
        self.batch_size = batch_size
        self.df = pd.DataFrame()
        self.epoch = 0
        
    def _reset_memory(self):
        self.memory = self._create_memory()
    
    @staticmethod
    def _create_memory():
        m = namedtuple("Memory", ["states", "actions", "rewards", "ep_lens", "ep_rewards", "weights"])
        for key in m._fields:
            setattr(m, key, [])
        return m
    
    def loss_fn(self, states, actions, rewards):
        log_ps = self.get_policy(states).log_prob(actions)
        return -(log_ps * rewards).mean()
    
    def get_policy(self, state):
        return Categorical(logits=self.model(state))
    
    def get_action(self, state):
        return self.get_policy(state).sample().item()
    
    def train(self, epochs, show_every=0):
        pbar = tqdm(range(epochs))
        data = []
        for epoch in pbar:
            if show_every and epoch and not (epoch % show_every): self.play()
            loss, returns, lens = self.train_step()
            row = pd.Series({
                "epoch": self.epoch,
                "loss": loss.item(),
                "max_return": max(returns),
                "max_len": max(lens),
                "avg_return": sum(returns) / len(returns),
                "avg_len": sum(lens) / len(lens),
            })
            self.df = self.df.append(row, ignore_index=True)
            pbar.set_postfix(row)
            self.epoch += 1
        return self.df
            
    def play(self, fps=30):
        state = self.env.reset()
        done = False
        ep_len = 0
        total_reward = 0
        while not done:
            if fps: self.env.render()
            action = self.get_action(torch.as_tensor(state, dtype=torch.float32))
            state, reward, done, _ = self.env.step(action)
            if fps: time.sleep(1/fps)
            
            total_reward += reward
            ep_len += 1
            
        if fps: self.env.close()
        return ep_len, total_reward
    
    def train_step(self):  
        while len(self.memory.states) < self.batch_size:
            state = self.env.reset()
            done = False
            total_reward = 0
            ep_len = 0
            ep_rewards = []
            while not done:
                self.memory.states.append(state.copy())
                action = self.get_action(torch.as_tensor(state, dtype=torch.float32))
                state, reward, done, _ = self.env.step(action)
                
                self.memory.actions.append(action)
                self.memory.rewards.append(reward)
                ep_rewards.append(reward)
                ep_len += 1
                total_reward += reward
                
            self.memory.ep_lens.append(ep_len)
            self.memory.ep_rewards.append(total_reward)
            self.memory.weights += list(reward_to_go(ep_rewards))
        
        self.optim.zero_grad()
        loss = self.loss_fn(
            torch.as_tensor(self.memory.states, dtype=torch.float32),
            torch.as_tensor(self.memory.actions, dtype=torch.int32),
            torch.as_tensor(self.memory.weights, dtype=torch.float32)
        )
        loss.backward()
        self.optim.step()

        ep_returns = self.memory.ep_rewards
        ep_lens = self.memory.ep_lens
        self._reset_memory()
        return loss, ep_returns, ep_lens
a = Agent(gym.make("MountainCar-v0"))

In [28]:
results = a.train(60, show_every=15)

base = alt.Chart(results.melt("epoch", value_vars=None, var_name="type"))
(
    base.mark_line(color="blue").encode(x="epoch:Q", y="value:Q", color="type:N")
)

alt.Chart(...)

In [103]:
a.play(fps=0)

(127, 127.0)